In [4]:
pip install pytest pyspark pandas statsmodels pmdarima


Note: you may need to restart the kernel to use updated packages.


In [54]:
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning

# Ignorar advertencias generales
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Ignorar advertencias de statsmodels específicas
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", message="A date index has been provided, but it has no associated frequency information")
warnings.filterwarnings("ignore", message="No supported index is available")


In [56]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, date_format, count
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima

# Configuro las rutas de Python necesarias para que Spark funcione correctamente
os.environ["PYSPARK_PYTHON"] = "C:\\Users\\ruben\\AppData\\Local\\Programs\\Python\\Python311\\python.exe"
os.environ["PYSPARK_DRIVER_PYTHON"] = "C:\\Users\\ruben\\AppData\\Local\\Programs\\Python\\Python311\\python.exe"

# Creo la sesión de Spark para poder trabajar con los datos
spark = SparkSession.builder \
    .appName("TestForecastWithCSV") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")


# Función que se encarga de generar el forecast, NO LA MODIFICO
def generar_forecast_por_asignatura(data):
    # Transformo la columna de timestamp para extraer solo la fecha
    data = data.withColumn("fecha", date_format(col("timestamp"), "yyyy-MM-dd"))

    # Agrupo los datos por estado de asistencia y fecha, y calculo totales diarios
    totales_diarios = data.groupBy("estado_asistencia", "fecha").agg(count("alumno").alias("total_diario"))
    totales_pandas = totales_diarios.toPandas()
    totales_pandas["fecha"] = pd.to_datetime(totales_pandas["fecha"])
    totales_pandas = totales_pandas.sort_values(by=["estado_asistencia", "fecha"])

    # Diccionario para guardar los resultados del forecast
    forecast_resultados = {"presentes": 0, "tarde": 0, "ausentes": 0}

    # Hago el análisis por cada estado (presente, tarde y ausente)
    for estado in ["presente", "tarde", "ausente"]:
        grupo_estado = totales_pandas[totales_pandas["estado_asistencia"] == estado]

        # Solo hago el forecast si hay más de 10 registros
        if len(grupo_estado) > 10:
            try:
                # Ajusto el modelo ARIMA
                serie = grupo_estado.set_index("fecha")["total_diario"]
                modelo_auto = auto_arima(serie, seasonal=False, stepwise=True, trace=False)
                modelo = ARIMA(serie, order=modelo_auto.order)
                ajuste = modelo.fit()
                prediccion = ajuste.forecast(steps=1)

                # Actualizo el diccionario con el valor predicho
                if estado == "presente":
                    forecast_resultados["presentes"] = round(prediccion.iloc[0])
                elif estado == "tarde":
                    forecast_resultados["tarde"] = round(prediccion.iloc[0])
                elif estado == "ausente":
                    forecast_resultados["ausentes"] = round(prediccion.iloc[0])
            except Exception as e:
                print(f"Error al ajustar ARIMA para {estado}: {str(e)}")

    return forecast_resultados

# Cargo el archivo CSV con los datos
df = spark.read.csv("C:/Users/ruben/Downloads/Bigggg/asistencia_clase_modificada.csv", header=True, inferSchema=True)

# Elimino los duplicados para asegurar que cada alumno esté solo una vez en cada clase
df = df.dropDuplicates(subset=["timestamp", "asignatura", "alumno"])

# Agrego una columna con solo la fecha para poder agrupar correctamente
df = df.withColumn("fecha", date_format(col("timestamp"), "yyyy-MM-dd"))

# Obtengo todas las asignaturas del dataset para analizarlas una por una
asignaturas = [row["asignatura"] for row in df.select("asignatura").distinct().collect()]

# Creo la función de test para procesar los datos y verificar el forecast
def test_generar_forecast_con_csv():
    print("Iniciando test con el CSV 'asistencia_clase_modificada.csv'...")

    # Diccionario para guardar los resultados del forecast de todas las asignaturas
    resultados_globales = {}

    # Proceso los datos por asignatura
    for asignatura in asignaturas:
        print(f"Procesando asignatura: {asignatura}")

        # Filtro los datos de la asignatura actual
        df_asignatura = df.filter(col("asignatura") == asignatura)

        # Llamo a la función de forecast para estos datos
        resultados = generar_forecast_por_asignatura(df_asignatura)

        # Guardo los resultados en el diccionario
        resultados_globales[asignatura] = resultados
        print(f"Resultados para {asignatura}: {resultados}")

    # Imprimo todos los resultados obtenidos
    print("Resultados Globales del Forecast:", resultados_globales)

    # Hago algunas validaciones básicas para asegurar que el forecast sea correcto
    for asignatura, resultados in resultados_globales.items():
        assert isinstance(resultados, dict), f"El resultado para {asignatura} debe ser un diccionario"
        assert "presentes" in resultados, f"Debe contener la clave 'presentes' para {asignatura}"
        assert "tarde" in resultados, f"Debe contener la clave 'tarde' para {asignatura}"
        assert "ausentes" in resultados, f"Debe contener la clave 'ausentes' para {asignatura}"
        assert resultados["presentes"] >= 0, f"El valor de 'presentes' no puede ser negativo para {asignatura}"
        assert resultados["tarde"] >= 0, f"El valor de 'tarde' no puede ser negativo para {asignatura}"
        assert resultados["ausentes"] >= 0, f"El valor de 'ausentes' no puede ser negativo para {asignatura}"

    print("Test completado exitosamente.")

# Llamo a la función de test para ejecutar todo
test_generar_forecast_con_csv()


Iniciando test con el CSV 'asistencia_clase_modificada.csv'...
Procesando asignatura: Grandes Volumenes de Datos
Resultados para Grandes Volumenes de Datos: {'presentes': 10, 'tarde': 6, 'ausentes': 4}
Procesando asignatura: Administracion de Sistemas
Resultados para Administracion de Sistemas: {'presentes': 10, 'tarde': 6, 'ausentes': 4}
Procesando asignatura: Ingenieria del Software
Resultados para Ingenieria del Software: {'presentes': 10, 'tarde': 6, 'ausentes': 4}
Procesando asignatura: Empresa y Legislacion
Resultados para Empresa y Legislacion: {'presentes': 10, 'tarde': 6, 'ausentes': 4}
Procesando asignatura: None
Resultados para None: {'presentes': 0, 'tarde': 0, 'ausentes': 0}
Resultados Globales del Forecast: {'Grandes Volumenes de Datos': {'presentes': 10, 'tarde': 6, 'ausentes': 4}, 'Administracion de Sistemas': {'presentes': 10, 'tarde': 6, 'ausentes': 4}, 'Ingenieria del Software': {'presentes': 10, 'tarde': 6, 'ausentes': 4}, 'Empresa y Legislacion': {'presentes': 10, 

In [58]:
def test_generar_forecast_con_csv():
    print("Iniciando test con el CSV 'asistencia_clase_modificada.csv'...")

    # Paso 1: Filtrar registros con asignaturas nulas o vacías
    print("Eliminando registros con asignaturas nulas o vacías...")
    df_filtrado = df.filter((col("asignatura").isNotNull()) & (col("asignatura") != ""))

    # Validar si hay registros después de filtrar
    total_registros = df_filtrado.count()
    print(f"Total de registros válidos después del filtrado: {total_registros}")
    assert total_registros > 0, "No hay registros válidos en el archivo después del filtrado."

    # Paso 2: Obtener asignaturas únicas
    asignaturas = [row["asignatura"] for row in df_filtrado.select("asignatura").distinct().collect()]
    print(f"Asignaturas únicas detectadas: {asignaturas}")
    assert len(asignaturas) > 0, "El archivo debe tener asignaturas válidas."

    # Paso 3: Procesar cada asignatura por separado
    print("Procesando cada asignatura por separado...")
    resultados_globales = {}

    for asignatura in asignaturas:
        print(f"Procesando asignatura: {asignatura}")

        # Filtro los datos por asignatura actual
        df_asignatura = df_filtrado.filter(col("asignatura") == asignatura)

        # Validar que hay datos suficientes para procesar
        registros_asignatura = df_asignatura.count()
        print(f"Total de registros para {asignatura}: {registros_asignatura}")
        assert registros_asignatura > 0, f"No hay registros para la asignatura {asignatura}."

        # Ejecutar forecast para esta asignatura
        resultados = generar_forecast_por_asignatura(df_asignatura)

        # Guardar resultados en el diccionario global
        resultados_globales[asignatura] = resultados
        print(f"Resultados para {asignatura}: {resultados}")

    # Paso 4: Validar los resultados globales
    print("Validando los resultados globales...")
    for asignatura, resultados in resultados_globales.items():
        assert isinstance(resultados, dict), f"El resultado para {asignatura} debe ser un diccionario"
        assert "presentes" in resultados, f"Debe contener la clave 'presentes' para {asignatura}"
        assert "tarde" in resultados, f"Debe contener la clave 'tarde' para {asignatura}"
        assert "ausentes" in resultados, f"Debe contener la clave 'ausentes' para {asignatura}"
        assert resultados["presentes"] >= 0, f"El valor de 'presentes' no puede ser negativo para {asignatura}"
        assert resultados["tarde"] >= 0, f"El valor de 'tarde' no puede ser negativo para {asignatura}"
        assert resultados["ausentes"] >= 0, f"El valor de 'ausentes' no puede ser negativo para {asignatura}"

    # Resultados finales
    print("Resultados Globales del Forecast:", resultados_globales)
    print("Test completado exitosamente.")

# Ejecutar el test
test_generar_forecast_con_csv()


Iniciando test con el CSV 'asistencia_clase_modificada.csv'...
Eliminando registros con asignaturas nulas o vacías...
Total de registros válidos después del filtrado: 9897
Asignaturas únicas detectadas: ['Grandes Volumenes de Datos', 'Administracion de Sistemas', 'Ingenieria del Software', 'Empresa y Legislacion']
Procesando cada asignatura por separado...
Procesando asignatura: Grandes Volumenes de Datos
Total de registros para Grandes Volumenes de Datos: 2438
Resultados para Grandes Volumenes de Datos: {'presentes': 10, 'tarde': 6, 'ausentes': 4}
Procesando asignatura: Administracion de Sistemas
Total de registros para Administracion de Sistemas: 2539
Resultados para Administracion de Sistemas: {'presentes': 10, 'tarde': 6, 'ausentes': 4}
Procesando asignatura: Ingenieria del Software
Total de registros para Ingenieria del Software: 2480
Resultados para Ingenieria del Software: {'presentes': 10, 'tarde': 6, 'ausentes': 4}
Procesando asignatura: Empresa y Legislacion
Total de registro